# Projeto 1 - MS629

Marcio Simões - 183360

In [1]:
import numpy as np
from sympy import *

In [2]:
# Defini os parametros
sig = 0.001
alp = 0.0001
gama = 0.5
the = 0.000001
beta = 0.001
M = 2500
epi = 0.1

## Função gradiente e hessiano

Definindo algumas funções gerais utilizadas em todos os métodos.

In [3]:
# Definindo as funções
def gradient_op(variables, f):
    return Matrix([Lambda(variables, f.diff(v)) for v in variables])

def hessiano_op(variables, f):
    return Matrix([[Lambda(variables, f.diff(v,u)) for u in variables] for v in variables])

In [4]:
# Funções para o cálculo
def gradient_res(grad_f, vetor):
    results = []
    for dx in grad_f:
        results.append(dx(*vetor))
    return results

def hessiano_res(hes_f, vetor):
    matrix = []
    lin = []
    arr = np.array(hes_f)
    for i in arr:
        for j in i:
            lin.append(j(*vetor))
        matrix.append(lin)
        lin = []
    return Matrix(matrix)

In [5]:
# Função para o cálculo de módulos
def modulo(x): 
    return float(sqrt(sum(i**2 for i in x)))

### Exemplos

Definindo uma função qualquer para teste dos cálculos anteriores.

In [6]:
# Defini variaveis
init_printing()
variables_ex = var('x1:3')

In [7]:
# Defini a função
f_ex = x1**4+2*x2*x1**2+x2**2-4*x1
fl_ex = Lambda(variables_ex, f_ex)
fl_ex

In [8]:
grad = gradient_op(variables_ex, f_ex)
grad

⎡               3              ⎤
⎢(x₁, x₂) ↦ 4⋅x₁  + 4⋅x₁⋅x₂ - 4⎥
⎢                              ⎥
⎢                  2           ⎥
⎣   (x₁, x₂) ↦ 2⋅x₁  + 2⋅x₂    ⎦

In [9]:
hes = hessiano_op(variables_ex, f_ex)
hes

⎡             ⎛    2     ⎞                 ⎤
⎢(x₁, x₂) ↦ 4⋅⎝3⋅x₁  + x₂⎠  (x₁, x₂) ↦ 4⋅x₁⎥
⎢                                          ⎥
⎣     (x₁, x₂) ↦ 4⋅x₁        (x₁, x₂) ↦ 2  ⎦

# Aplicação dos Algoritmos

Para tal, foi construído uma função para cada um dos algoritmos: Gradiente (GR), Newton Globalizado (NG)
e Secante Globalizado (DFP). Onde: 

- as entradas são: *variaveis, função a ser minimizada, x0, epson (ou precisão)*
- as saídas são: *x mínimo, valor mínimo da função, número de iterações*.

## Gradiente

In [10]:
# Função que aplica o método
def GR(variables, f, x0, epi):
    fl = Lambda(variables, f)
    grad = gradient_op(variables, f)
    
    k = 0
    x = x0
    while modulo(gradient_res(grad, x)) > epi and k < M:
        grad_l = np.array(gradient_res(grad, x), dtype=np.float)
        d = -grad_l
        
        #calculando xk
        t = 1
        while fl(*(x+d*t)) > fl(*x) + alp*t*sum(d*grad_l):
            t = gama*t
    
        x = x+d*t
        k = k + 1
    return x, fl(*x), k

### Exemplo com GR

In [11]:
# Defini x0
x0 = [1,0]

In [12]:
# Aplica o Gradiente
GR(variables_ex, f_ex, x0, 0.01)

(array([  5.20131178, -26.82939042]), -20.7549573475399, 2500)

#### Função Quadrática com GR

In [13]:
init_printing()
variables = var('x1:6')
f = x1**2+2*x2**2+3*x3**2+4*x4**2+5*x5**2
fl = Lambda(variables, f)
fl

In [14]:
# Defini x0
x0 = [1,1,1,1,1]

In [15]:
# Aplica o Gradiente
GR(variables, f, x0, 0.01)

(array([ 2.08568573e-03,  0.00000000e+00,  9.53674316e-07,  0.00000000e+00,
        -6.95228577e-04]),
 6.76680156175280e-6,
 13)

In [16]:
# Aplica o Gradiente
GR(variables, f, x0, 0.00001)

(array([ 2.17504248e-06,  0.00000000e+00, -4.54747351e-13,  0.00000000e+00,
        -7.25014161e-07]),
 7.35903746516739e-12,
 27)

#### Rosenbrook com GR

In [17]:
init_printing()
variables = var('x1:5')
f = 100*(x2-x1**2)**2+(x1-1)**2+100*(x4-x3**2)**2+(x3-1)**2
fl = Lambda(variables, f)
fl

In [18]:
# Defini x0
x0 = [0,0,0,0]

In [19]:
# Aplica o Gradiente
GR(variables, f, x0, 0.01)

(array([0.98150242, 0.96331717, 0.98150242, 0.96331717]),
 0.000684498661681655,
 2500)

In [20]:
# Aplica o Gradiente
GR(variables, f, x0, 0.001)

(array([0.98150242, 0.96331717, 0.98150242, 0.96331717]),
 0.000684498661681655,
 2500)

#### Styblinsky–Tang com GR

In [21]:
init_printing()
variables = var('x1:4')
f = x1**4-16*x1**2+5*x1+x2**4-16*x2**2+5*x2+x3**4-16*x3**2+5*x3
fl = Lambda(variables, f)
fl

In [22]:
# Defini x0
x0 = [0,0,0]

In [23]:
# Aplica o Gradiente
GR(variables, f, x0, 0.01)

(array([-2.90355406, -2.90355406, -2.90355406]), -234.996994180997, 6)

#### Rastrigin com GR

In [24]:
init_printing()
variables = var('x1:4')
f = x1**2 - 10*cos(2*pi*x1)+x2**2 - 10*cos(2*pi*x2)+x3**2 - 10*cos(2*pi*x3)
fl = Lambda(variables, f)
fl

In [25]:
# Defini x0
x0 = [0,0,0]

In [26]:
GR(variables, f, x0, 0.01)

In [27]:
# Defini x0
x0 = [5,5,5]

In [28]:
GR(variables, f, x0, 0.01)

(array([0., 0., 0.]), -30, 1)

## Newton Globalizado

In [29]:
# Cálculo da direção
def direcao_NG(variables, beta, grad_l, hes_l):
    mi = 0
    A = []
    while True:
        A = hes_l + mi*np.identity(len(variables))
        try:
            d = np.linalg.solve(A, -grad_l)
        
            if sum(d*grad_l) > -the*modulo(grad_l)*modulo(d):
                mi = max(2*mi, beta)
            else:
                break
        except:
            mi = max(2*mi, beta)
    return d

In [30]:
# Função que aplica o método
def NG(variables, f, x0, epi):
    fl = Lambda(variables, f)
    grad = gradient_op(variables, f)
    hes = hessiano_op(variables, f)
    
    k = 0
    x = x0
    while modulo(gradient_res(grad, x)) > epi and k < M:
        grad_l = np.array(gradient_res(grad, x), dtype=np.float)
        hes_l = np.array(hessiano_res(hes, x), dtype=np.float)
    
        d = direcao_NG(variables, beta, grad_l, hes_l)
            
        #fazendo as verificações de armijo
        if modulo(d) < sig*modulo(grad_l):
            d = the*(modulo(grad_l)/modulo(d))*d
        
        #calculando xk
        t = 1
        while fl(*(x+d*t)) > fl(*x) + alp*t*sum(d*grad_l):
            t = gama*t
    
        x = x+d*t
        k = k + 1
    return x, fl(*x), k

### Exemplo com NG

In [31]:
# Defini x0
x0 = [1,0]

In [32]:
# Aplica o Newton Globalizado
NG(variables_ex, f_ex, x0, 0.01)

(array([    912.44752704, -832559.71010927]), -3649.18237304688, 2500)

#### Função Quadrática com NG

In [33]:
init_printing()
variables = var('x1:6')
f = x1**2+2*x2**2+3*x3**2+4*x4**2+5*x5**2
fl = Lambda(variables, f)
fl

In [34]:
# Defini x0
x0 = [1,1,1,1,1]

In [35]:
# Aplica o Newton Globalizado
NG(variables, f, x0, 0.01)

(array([0., 0., 0., 0., 0.]), 0, 1)

In [36]:
# Aplica o Newton Globalizado
NG(variables, f, x0, 0.0001)

(array([0., 0., 0., 0., 0.]), 0, 1)

#### Rosenbrook com NG

In [37]:
init_printing()
variables = var('x1:5')
f = 100*(x2-x1**2)**2+(x1-1)**2+100*(x4-x3**2)**2+(x3-1)**2
fl = Lambda(variables, f)
fl

In [38]:
# Defini x0
x0 = [0,0,0,0]

In [39]:
# Aplica o Newton Globalizado
NG(variables, f, x0, 0.01)

(array([0.99995223, 0.99990215, 0.99995223, 0.99990215]),
 5.63790109091734e-9,
 12)

In [40]:
NG(variables, f, x0, 0.000001)

(array([1., 1., 1., 1.]), 6.84151946004567e-28, 14)

#### Styblinsky–Tang com NG

In [41]:
init_printing()
variables = var('x1:4')
f = x1**4-16*x1**2+5*x1+x2**4-16*x2**2+5*x2+x3**4-16*x3**2+5*x3
fl = Lambda(variables, f)
fl

In [42]:
# Defini x0
x0 = [0,0,0]

In [43]:
NG(variables, f, x0, 0.01)

(array([-2.90353471, -2.90353471, -2.90353471]), -234.996994222581, 4)

#### Rastrigin com NG

In [44]:
init_printing()
variables = var('x1:4')
f = x1**2 - 10*cos(2*pi*x1)+x2**2 - 10*cos(2*pi*x2)+x3**2 - 10*cos(2*pi*x3)
fl = Lambda(variables, f)
fl

In [45]:
# Defini x0
x0 = [0,0,0]

In [46]:
NG(variables, f, x0, 0.01)

In [47]:
# Defini x0
x0 = [5,5,5]

In [48]:
NG(variables, f, x0, 0.00001)

(array([4.9746914, 4.9746914, 4.9746914]),
 74.2426634715323 - 30*cos(1.94938279302003*pi),
 2)

In [49]:
74.2458269771614 - 30*cos(1.94959476408705*3.14)

## Secante Globalizado

In [50]:
# Função que aplica o método
def DFP(variables, f, x0, epi):
    fl = Lambda(variables, f)
    grad = gradient_op(variables, f)
    
    k = 0
    x = x0
    H = np.identity(len(variables))
    grad_l = np.array(gradient_res(grad, x), dtype=np.float)
    while modulo(gradient_res(grad, x)) > epi and k < M:
        d = -H*np.matrix(grad_l).T
        d = d.T.tolist()[0]

        if sum(d*grad_l) > -the*modulo(grad_l)*modulo(d):
            d = -grad_l
            H = np.identity(len(variables))
            
        #fazendo as verificações de armijo 
        if modulo(d) < sig*modulo(grad_l):
            d = the*(modulo(grad_l)/modulo(d))*np.array(d)
        
        #calculando xk
        t = 1
        p = x
        while fl(*(x+np.array(d)*t)) > fl(*x) + alp*t*sum(d*grad_l):
            t = gama*t
        x = x+np.array(d)*t
        
        grad_lprox = np.array(gradient_res(grad, x), dtype=np.float)
        p = x - p
        q = grad_lprox - grad_l
        r = np.matrix(H)*(np.matrix(q).T)
        
        if sum(np.array(p)*np.array(q)) > 0:
            H = H + np.matrix(p).T*np.matrix(p)/(np.matrix(p)*np.matrix(q).T) - r*r.T/(r.T*np.matrix(q).T) 
     
        grad_l = grad_lprox
        k = k + 1
    return x, fl(*x), k

### Exemplo DFP

In [51]:
# Defini x0
x0 = [1,1]

In [52]:
# Aplica o DFP
DFP(variables_ex, f_ex, x0, 0.01)

(array([   59.31044278, -3516.91049434]), -236.572437439114, 2500)

#### Função Quadrática com DFP

In [53]:
init_printing()
variables = var('x1:6')
f = x1**2+2*x2**2+3*x3**2+4*x4**2+5*x5**2
fl = Lambda(variables, f)
fl

In [54]:
# Defini x0
x0 = [1,1,1,1,1]

In [55]:
DFP(variables, f, x0, 0.01)

(array([-2.07734013e-05, -4.39438212e-05, -1.04099647e-05, -1.47166860e-05,
         1.37994415e-05]),
 6.43720145521847e-9,
 7)

In [56]:
DFP(variables, f, x0, 0.00001)

(array([ 1.46986205e-07,  4.65781158e-07,  3.56589416e-07,  3.10545111e-07,
        -2.84068801e-07]),
 1.62620563744219e-12,
 8)

#### Rosenbrook com DFP

In [57]:
init_printing()
variables = var('x1:5')
f = 100*(x2-x1**2)**2+(x1-1)**2+100*(x4-x3**2)**2+(x3-1)**2
fl = Lambda(variables, f)
fl

In [58]:
# Defini x0
x0 = [0,0,0,0]

In [59]:
DFP(variables, f, x0, 0.01)

(array([0.99851741, 0.99701941, 0.99851741, 0.99701941]),
 4.45809932426327e-6,
 50)

In [60]:
DFP(variables, f, x0, 0.00001)

(array([0.99999999, 0.99999998, 0.99999999, 0.99999998]),
 3.48177057085698e-16,
 60)

#### Styblinsky–Tang com DFP

In [61]:
init_printing()
variables = var('x1:4')
f = x1**4-16*x1**2+5*x1+x2**4-16*x2**2+5*x2+x3**4-16*x3**2+5*x3
fl = Lambda(variables, f)
fl

In [62]:
# Defini x0
x0 = [0,0,0]

In [63]:
DFP(variables, f, x0, 0.01)

(array([-2.90353278, -2.90353278, -2.90353278]), -234.996994222467, 6)

#### Rastrigin com DFP

In [64]:
init_printing()
variables = var('x1:4')
f = x1**2 - 10*cos(2*pi*x1)+x2**2 - 10*cos(2*pi*x2)+x3**2 - 10*cos(2*pi*x3)
fl = Lambda(variables, f)
fl

In [65]:
# Defini x0
x0 = [0,0,0]

In [66]:
DFP(variables, f, x0, 0.01)

In [67]:
# Defini x0
x0 = [5,5,5]

In [68]:
DFP(variables, f, x0, 0.01)

(array([0., 0., 0.]), -30, 1)